# Teti Programming Week - Data Science Competition

## Kelompok: yamaap

Anggota: 
- Rayhan Adi Wicaksono
- Laily Medha Nur Ihsanti
- Anisa Bintang Maharani

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')

df_country = pd.read_csv('./dataset/country.csv')
df_city = pd.read_csv('./dataset/worldcities.csv')

In [3]:
df_train.head()

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,This is a new hotel. Great staff. Loved intera...,en,8.0
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,Lovely attentive and welcoming staff Clean an...,en,10.0
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,Cozy space to chill \n Not much variety of fo...,en,7.1
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,The hotel has certainly benefited from the inv...,en,10.0
4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,Excellent location for the Albert Hall and pl...,en,10.0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278435 entries, 0 to 278434
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   hotel_name       278435 non-null  object 
 1   hotel_location   270093 non-null  object 
 2   review_id        278435 non-null  object 
 3   review_date      278277 non-null  object 
 4   review_text      278434 non-null  object 
 5   review_language  278384 non-null  object 
 6   review_rating    278435 non-null  float64
dtypes: float64(1), object(6)
memory usage: 14.9+ MB


In [6]:
df_train.describe()

,review_rating
count,278435.000000
mean,8.427491
std,1.666610
min,2.000000
25%,7.500000
50%,8.800000
75%,10.000000
max,10.000000


In [7]:
df_train.describe(include=['object'])

,hotel_name,hotel_location,review_id,review_date,review_text,review_language
count,278435,270093,278435,278277,278434,278384
unique,2434,733,278435,4339,271354,39
top,The Savoy,London United Kingdom,3mcTZLLqVE5ztNyE,02-08-2017,Location,en
freq,3007,76208,1,1317,522,270961


# Data Understanding

## review_language

### review_language unique category

In [8]:
df_train['review_language'].unique()

array(['en', 'da', 'es', 'it', 'fr', 'de', 'ca', 'pt', 'ro', 'af', 'tr',
       'hu', 'pl', 'ko', 'hr', 'nl', 'no', 'et', 'vi', 'cy', 'so', 'sl',
       'tl', 'sw', nan, 'id', 'sv', 'fi', 'ja', 'lv', 'cs', 'sk', 'sq',
       'fa', 'ru', 'ar', 'lt', 'zh-cn', 'he', 'th'], dtype=object)

Terlihat bahwa review yang terdapat dari dataset menggunakan berbagai bahasa yang berbeda. Karena berdasarkan describe, bahasa inggris merupakan bahasa yang paling banyak digunakan, yaitu oleh 270k+ review, ada dua opsi yang bisa digunakan, yaitu:
- Pake data b ing saja
- Translate bahasa lain ke bahasa inggris

## hotel_location

Diketahui bahwa hotel_location ditulis dengan format 'Kota Negara', maka dari itu, untuk saat ini, kita pisahkan terlebih dahulu kota dengan negara. Metode pemisahannya split sederhana dulu, kedepannya bisa diimprove karena bisa aja nama kota lebih dari satu kata.

In [9]:
df_city.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [10]:
# Create a function to identify and split the country from location
def split_country(row):
    location = row['hotel_location']
    for country in df_country['value']:
        if country in location:
            return location.split(country)[0].strip(), country

    return location, None
    
# Fill NaN values in 'hotel_location' column with an empty string
df_train['hotel_location'].fillna('', inplace=True)

# Apply the function to split 'Location' into 'City' and 'Country' columns
df_train[['hotel_city', 'hotel_country']] = df_train.apply(split_country, axis=1, result_type='expand')

In [11]:
# Create a function to identify and split the city from recent city
def split_city(row):
    location = row['hotel_city']
    for city in df_city['city']:
        if city in location:
            return location.split(city)[0].strip(), city

    return location, None
    
# Fill NaN values in 'hotel_location' column with an empty string
df_train['hotel_city'].fillna('', inplace=True)

# Apply the function to split 'Location' into 'City' and 'Country' columns
df_train[['hotel_city', 'dump']] = df_train.apply(split_city, axis=1, result_type='expand')
df_train.drop('dump', axis = 1)

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating,hotel_city,hotel_country
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,This is a new hotel. Great staff. Loved intera...,en,8.0,Eads,US
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,Lovely attentive and welcoming staff Clean an...,en,10.0,,United Kingdom
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,Cozy space to chill \n Not much variety of fo...,en,7.1,,United Kingdom
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,The hotel has certainly benefited from the inv...,en,10.0,,United Kingdom
4,Millennium Gloucester Hotel London,,Fo44un6pn2M7XoVJ,23-10-2015,Excellent location for the Albert Hall and pl...,en,10.0,,None
...,...,...,...,...,...,...,...,...,...
278430,Hilton London Euston,London United Kingdom,Oabt4y5utR4burvS,16-02-2016,The room we had on the ground floor was one o...,en,10.0,,United Kingdom
278431,Le Burgundy Paris,Paris France,t5xwremU01FrczxF,13-08-2015,Everyday there was a sweet sent to the room I...,en,7.9,,France
278432,Best Western Premier Kaiserhof Wien,Wieden Austria,jNFRId7DR7d8AGO1,25-10-2015,The staff are outstanding We arrived early in...,en,10.0,Wieden,Austria
278433,Novotel Paris Gare De Lyon,Paris France,vqcbMzXqiSuXRHMU,29-10-2015,Great staff\n Queue for breakfast,en,9.6,,France


In [12]:
df_train.hotel_city.value_counts()

                 259991
Marsh 9SJ          2382
Newham             1309
Devonshire         1125
Wieden             1062
                  ...  
Shandaken             1
Taos                  1
Saint Germain         1
Prosser               1
Cosby                 1
Name: hotel_city, Length: 177, dtype: int64

In [13]:
df_train.hotel_country.value_counts()

United Kingdom    140228
Spain              29929
France             25959
Austria            18853
Italy              18573
Netherlands        17258
US                  5037
Mexico               779
Switzerland          342
Panama               192
Nicaragua            186
India                  5
Chad                   2
Cuba                   1
Grenada                1
Name: hotel_country, dtype: int64

Berikut merupakan persebarannya, note: London tampaknya masih belum semua

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from googletrans import Translator
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
pd.set_option('display.max_rows', None)
import hunspell
import unicodedata
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [2]:
df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')


In [3]:
df_train.review_text[1200]

' Both the food and the location were wonderful The staff was very helpful and quick to comply all of our requests We ll definitely be coming back again '

In [4]:
df_train.head(10)

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,This is a new hotel. Great staff. Loved intera...,en,8.0
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,Lovely attentive and welcoming staff Clean an...,en,10.0
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,Cozy space to chill \n Not much variety of fo...,en,7.1
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,The hotel has certainly benefited from the inv...,en,10.0
4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,Excellent location for the Albert Hall and pl...,en,10.0
5,Radisson Blu Champs Elys es Paris,Paris France,u5GIYVDPKzyelPUE,14-11-2015,No Positive\n The sink was not practical The g...,en,7.9
6,The Cumberland A Guoman Hotel,London 7DL United Kingdom,KMQ7WwisR6N9ifT9,05-06-2017,Friendly staff particularly in the bar\n Room...,en,9.2
7,Hotel Bristol,Milan Italy,abRtDdSsOHIf2j0S,09-07-2016,No Positive\n The kitchen staff was non cooper...,en,7.1
8,Grange Tower Bridge Hotel,London United Kingdom,tYLpCYokKtyShCCV,01-03-2017,No Positive\n The fact food was not included i...,en,10.0
9,Park Plaza Victoria London,London United Kingdom,EvMXR91LcGbAUbdr,01-01-2017,Didn t have breakfast,da,8.8


# Text Data Preprocessing

## Fillna

In [5]:
# Fill NaN values in 'hotel_location' column with an empty string

def fill_null_value(df):
    df['review_text'].fillna('', inplace=True)
    df['review_language'].fillna('en', inplace=True)
    return df

df_train = fill_null_value(df_train)
df_test = fill_null_value(df_test)

## Translate to English

Disini kita import preprocessed data saja

In [6]:
non_english_df_train = pd.read_csv('./dataset/train_to_english.csv')
non_english_df_train.set_index('Unnamed: 0', inplace=True)
non_english_df_test = pd.read_csv('./dataset/test_to_english.csv')
non_english_df_test.set_index('Unnamed: 0', inplace=True)

In [7]:
def translate_to_english_preprocessed(df, non_english_df):
    for index_value in non_english_df.index:
        df.loc[df.index == index_value, 'review_text'] = non_english_df.loc[non_english_df.index == index_value, 'review_text'].values


translate_to_english_preprocessed(df_train, non_english_df_train)
translate_to_english_preprocessed(df_test, non_english_df_test)

## Handle Contractions

In [8]:
contractions_dict = {
    "ain't": "are not",
    "'s": " is",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "that'd": "that would",
    "that'd've": "that would have",
    "there'd": "there would",
    "there'd've": "there would have",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "u": "you",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what've": "what have",
    "when've": "when have",
    "where'd": "where did",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who've": "who have",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}


In [9]:
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

def handle_contractions(df):
    for i in tqdm(range(len(df['review_text']))):
        value = str(df['review_text'][i])
        value_handled_contractions = expand_contractions(value)
        df.loc[i, "review_text"] = value_handled_contractions

# Assuming df_train and df_test are DataFrames with a 'review_text' column
handle_contractions(df_train)
handle_contractions(df_test)

100%|█████████████████████████████████████████████████████████████████████████| 278436/278436 [00:30<00:00, 9115.85it/s]


## Drop Link

Sementara tidak dipakai

In [10]:
link_pattern = r'\s*https?://\S+$'

def drop_link(df):
    for i in tqdm(range(len(df['review_text']))):
        # Perform the desired manipulation
        value = str(df['review_text'][i])
        
        output = re.sub(link_pattern, '', value)
        
        #print(output)
        # Change the value in place
        df.loc[i,"review_text"]= output

# drop_link(df_train)
# drop_link(df_test)

## Drop Escape Character

In [11]:
def remove_escape_sequences(text):
    # Remove escape sequences and multiple dots
    text = re.sub(r'\\\\', '', text)     # \\
    text = re.sub(r"\\'", "'", text)     # \'
    text = re.sub(r'\\"', '"', text)     # \"
    text = re.sub(r'\\n', '', text)      # \n
    text = re.sub(r'\n\n', '', text)
    text = re.sub(r'\\t', '', text)      # \t
    text = re.sub(r'\\b', '', text)      # \b
    text = re.sub(r'\\r', '', text)      # \r
    text = re.sub(r'\\f', '', text)      # \f
    text = re.sub(r'\\012', '', text)    # \012
    text = re.sub(r'\\x0A', '', text)    # \x0A
    text = re.sub(r'\.{2,}', '.', text)
    
    return text

def drop_escape_sequences(df):
    for i in tqdm(range(len(df['review_text']))):
        # Perform the desired manipulation
        value = str(df['review_text'][i])
        value = remove_escape_sequences(value)
        df.loc[i,"review_text"]= value

drop_escape_sequences(df_train)
drop_escape_sequences(df_test)

100%|████████████████████████████████████████████████████████████████████████| 278436/278436 [00:23<00:00, 11787.23it/s]


In [12]:
df_train.head()

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,This is a new hotel. Great staff. Loved intera...,en,8.0
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,Lovely attentive and welcoming staff Clean an...,en,10.0
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,Cozy space to chill \n Not myouch variety of ...,en,7.1
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,The hotel has certainly benefited from the inv...,en,10.0
4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,Excellent location for the Albert Hall and pl...,en,10.0


## Drop Punctuation

In [13]:
def remove_punctuation(sentence):
    # Create a translation table mapping punctuation characters to None
    punctuation_to_remove = string.punctuation + '‘’“”—€'
    translator = str.maketrans("", "", string.punctuation)
    
    # Remove punctuation using the translation table
    sentence_without_punctuation = sentence.translate(translator)
    
    return sentence_without_punctuation

def drop_punctuation(df):
    for i in tqdm(range(len(df['review_text']))):
        value = str(df['review_text'][i])
        output =  remove_punctuation(value)
        # Change the value in place
        df.loc[i,"review_text"]= output

drop_punctuation(df_train)
drop_punctuation(df_test)

100%|████████████████████████████████████████████████████████████████████████| 278436/278436 [00:21<00:00, 12931.90it/s]


## Drop Emoticon

In [14]:
# Iterate over the rows in the dataframe
emoticon_pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F900-\U0001F9FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F300-\U0001F5FF\U0001F900-\U0001F9FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F900-\U0001F9FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F300-\U0001F5FF\U0001F900-\U0001F9FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U00002600-\U000026FF\U00002700-\U000027BF\U0001F300-\U0001F5FF\U0001F900-\U0001F9FF]+'

def drop_emoticon(df):
    for i in tqdm(range(len(df['review_text']))):
        # Perform the desired manipulation
        
        value = str(df['review_text'][i])
        
        output = re.sub(emoticon_pattern, '', value)
        
        #print(output)
        # Change the value in place
        df.loc[i,"review_text"]= output
        
drop_emoticon(df_train)
drop_emoticon(df_test)


100%|████████████████████████████████████████████████████████████████████████| 278436/278436 [00:22<00:00, 12380.41it/s]


## Lowercasing

In [15]:
def lowercasing(df):
    for i in tqdm(range(len(df['review_text']))):
        # Perform the desired manipulation
        
        value = str(df['review_text'][i])
        
        output =  value.lower()

        # Change the value in place
        df.loc[i,"review_text"]= output
        
lowercasing(df_train)
lowercasing(df_test)


100%|████████████████████████████████████████████████████████████████████████| 278436/278436 [00:20<00:00, 13879.25it/s]


In [16]:
# Checkpoint

df_train.to_csv('train_lowercased_v2.csv', index = True)
df_test.to_csv('test_lowercased_v2.csv', index = True)

df_train.head()

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,this is a new hotel great staff loved interact...,en,8.0
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,lovely attentive and welcoming staff clean an...,en,10.0
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,cozy space to chill \n not myouch variety of ...,en,7.1
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,the hotel has certainly benefited from the inv...,en,10.0
4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,excellent location for the albert hall and pl...,en,10.0


## Tokenization

In [2]:
df_train = pd.read_csv('./dataset/train_lowercased_v2.csv')
df_test = pd.read_csv('./dataset/test_lowercased_v2.csv')

In [3]:
def tokenization(df):
    df['review_text'] = df['review_text'].apply(lambda x: nltk.word_tokenize(str(x)))

tokenization(df_train)
tokenization(df_test)

In [4]:
df_train.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,"[this, is, a, new, hotel, great, staff, loved,...",en,8.0
1,1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,"[lovely, attentive, and, welcoming, staff, cle...",en,10.0
2,2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,"[cozy, space, to, chill, not, myouch, variety,...",en,7.1
3,3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,"[the, hotel, has, certainly, benefited, from, ...",en,10.0
4,4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,"[excellent, location, for, the, albert, hall, ...",en,10.0


## Stemming

In [5]:
ps = PorterStemmer()

def stemming(df):
    for i in tqdm(range(len(df['review_text']))):
        # Perform the desired manipulation
        value = df.at[i, 'review_text']  # Assuming review_text is a list of words

        # Perform stemming on each word in the list
        stemmed_text = [ps.stem(word) for word in value]

        # Change the value in place
        df.at[i, 'review_text'] = stemmed_text

# stemming(df_train)
# stemming(df_test)

## Stemming V2: with hunspell

In [6]:
hobj = hunspell.HunSpell('./dataset/en_US.dic', './dataset/en_US.aff')

def stemming(df):
    for i in tqdm(range(len(df))):
        value = df.at[i, 'review_text']

        stemmed_text = []
        for word in value:
            try:
                word_str = word.decode('utf-8')  # Decode bytes to string
                stemmed_word = hobj.stem(word_str)
                stemmed_text.append(stemmed_word)
            except UnicodeEncodeError:
                # Handle exception caused by specific characters
                # print(f"UnicodeEncodeError for word: {word}")
                # Add the original word without stemming
                stemmed_text.append(word_str)
            except AttributeError:
                # If it's already a string, just append it
                stemmed_text.append(word)
        
        # Change the value in place
        df.at[i, 'review_text'] = stemmed_text
        
        # Change the value in place
        df.at[i, 'review_text'] = stemmed_text

stemming(df_train)
stemming(df_test)

100%|████████████████████████████████████████████████████████████████| 278436/278436 [00:18<00:00, 15079.33it/s]


## Lemmatization

Note: ada kemungkinan lemmatization jelek, jd nanti coba gapake dulu aja

In [7]:
df_train.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,"[this, is, a, new, hotel, great, staff, loved,...",en,8.0
1,1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,"[lovely, attentive, and, welcoming, staff, cle...",en,10.0
2,2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,"[cozy, space, to, chill, not, myouch, variety,...",en,7.1
3,3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,"[the, hotel, has, certainly, benefited, from, ...",en,10.0
4,4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,"[excellent, location, for, the, albert, hall, ...",en,10.0


In [8]:
lemmatizer = WordNetLemmatizer()

def lemmatization(df):
    lemmatized_texts = []  # To store lemmatized texts
    
    for i in tqdm(range(len(df['review_text']))):
        value = df['review_text'][i]
        
        # Perform lemmatization on each word in the list
        output = [lemmatizer.lemmatize(word) for word in value]
        
        # Append the lemmatized list to the new list
        lemmatized_texts.append(output)
    
    # Assign the new list to the DataFrame column
    df['review_text'] = lemmatized_texts

lemmatization(df_train)
lemmatization(df_test)

100%|█████████████████████████████████████████████████████████████████| 278436/278436 [00:30<00:00, 9068.62it/s]


In [9]:
df_train.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,"[this, is, a, new, hotel, great, staff, loved,...",en,8.0
1,1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,"[lovely, attentive, and, welcoming, staff, cle...",en,10.0
2,2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,"[cozy, space, to, chill, not, myouch, variety,...",en,7.1
3,3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,"[the, hotel, ha, certainly, benefited, from, t...",en,10.0
4,4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,"[excellent, location, for, the, albert, hall, ...",en,10.0


In [12]:
df_test.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language
0,0,Ohla Barcelona,Barcelona Spain,9p62eIN5NkEadRe7,18-03-2017,"[no, positive, the, shower, wa, open, plan, to...",en
1,1,Radisson Blu Edwardian Mercer Street,London United Kingdom,WfQozacJ1lzXMFQg,12-02-2017,"[the, staff, made, yous, feel, welcome, a, soo...",en
2,2,The May Fair Hotel,London United Kingdom,oahzyhSR3ZNHJBuI,06-01-2017,"[great, location, being, jyoust, off, piccadil...",en
3,3,Park Plaza County Hall London,United Kingdom,lcO2pUMVQ09RmbHZ,07-02-2017,"[good, personel, and, smiling, face, additiona...",en
4,4,Best Western Premier Hotel Couture,Delflandlaan Amsterdam Netherlands,DmMu1z2SozPxykUS,26-11-2015,"[we, had, a, wonderfyoul, corner, room, very, ...",en


## Remove Stopwords

In [11]:
df_train_backup = df_train.copy()
df_test_backup = df_test.copy()

In [27]:
df_train = df_train_backup.copy()
df_test = df_test_backup.copy()

In [27]:
# Assuming 'review_text' is the column containing your text data
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    filtered_text = [word for word in text if word.lower() not in stop_words]
    return filtered_text

# Function to remove stopwords from the 'review_text' column
def drop_stopwords(df):
    for i in tqdm(range(len(df['review_text']))):
        value = df['review_text'][i]  # No need for str() here
        
        # Perform lemmatization on each word in the list
        output = remove_stopwords(value)
        
        # Replace the list in the DataFrame with a string representation of the list
        df.loc[i, "review_text"] = output

# drop_stopwords(df_train)
# drop_stopwords(df_test)

100%|████████████████████████████████████████████████████████████████████████| 278436/278436 [00:24<00:00, 11158.36it/s]


## Remove Stopword V2 with Scapy

In [40]:

nlp = English()

def remove_stopwords(text):
    filtered_text = [word for word in text if nlp.vocab[word].is_stop == False]
    return filtered_text

# Function to remove stopwords from the 'review_text' column
def drop_stopwords(df):
    dropped_stopword_texts = []  # To store new array texts
    
    for i in tqdm(range(len(df['review_text']))):
        value = df['review_text'][i]  # No need for str() here
        
        # Perform lemmatization on each word in the list
        output = remove_stopwords(value)

        dropped_stopword_texts.append(output)
        
    # Replace the list in the DataFrame with a string representation of the list
    df["review_text"] = dropped_stopword_texts

drop_stopwords(df_train)
drop_stopwords(df_test)


100%|████████████████████████████████████████████████████████████████| 278436/278436 [00:10<00:00, 26762.01it/s]


In [41]:
df_train.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,"[new, hotel, great, staff, loved, interacting,...",en,8.0
1,1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,"[lovely, attentive, welcoming, staff, clean, c...",en,10.0
2,2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,"[cozy, space, chill, myouch, variety, food, av...",en,7.1
3,3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,"[hotel, ha, certainly, benefited, investment, ...",en,10.0
4,4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,"[excellent, location, albert, hall, place, eat...",en,10.0


In [42]:
df_test.head()

,Unnamed: 0,hotel_name,hotel_location,review_id,review_date,review_text,review_language
0,0,Ohla Barcelona,Barcelona Spain,9p62eIN5NkEadRe7,18-03-2017,"[positive, shower, wa, open, plan, room, bedro...",en
1,1,Radisson Blu Edwardian Mercer Street,London United Kingdom,WfQozacJ1lzXMFQg,12-02-2017,"[staff, yous, feel, welcome, soon, stepped, th...",en
2,2,The May Fair Hotel,London United Kingdom,oahzyhSR3ZNHJBuI,06-01-2017,"[great, location, jyoust, piccadilly, byout, r...",en
3,3,Park Plaza County Hall London,United Kingdom,lcO2pUMVQ09RmbHZ,07-02-2017,"[good, personel, smiling, face, additional, be...",en
4,4,Best Western Premier Hotel Couture,Delflandlaan Amsterdam Netherlands,DmMu1z2SozPxykUS,26-11-2015,"[wonderfyoul, corner, room, good, room, rate, ...",en


In [43]:
df_train.review_text[0][1]

'hotel'

# Tabular Data Preprocessing

## Bikin rerata untuk nama hotel yang sama

In [44]:
# Calculate the mean review rating for each hotel
df_train['hotel_rating_mean'] = df_train.groupby('hotel_name')['review_rating'].transform('mean')

# Calculate mean review rating for each hotel in df_train
mean_ratings = df_train.groupby('hotel_name')['review_rating'].mean().reset_index()

# Merge mean values into df_test
df_test = pd.merge(df_test, mean_ratings, on='hotel_name', how='left')
df_test = df_test.rename(columns={'review_rating': 'hotel_rating_mean'})

## Save to file

V1: with stemming without lemmatization
V2: without stemming with lemmatization
V3: using stemming and lemmatization
V5: Improved Stemming + a lot
V6: Use Improved Stopword

In [45]:
df_train.to_csv('dataset/train_preprocessed_v6.csv', index = True)
df_test.to_csv('dataset/test_preprocessed_v6.csv', index = True)

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from scipy.sparse import hstack 

In [2]:
df_train = pd.read_csv("./dataset/train_preprocessed_v4.csv")
df_test = pd.read_csv("./dataset/test_preprocessed_v4.csv")

df = pd.concat([df_train, df_test], axis=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556871 entries, 0 to 278435
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0.2       556871 non-null  int64  
 1   Unnamed: 0.1       556871 non-null  int64  
 2   Unnamed: 0         556871 non-null  int64  
 3   hotel_name         556871 non-null  object 
 4   hotel_location     540465 non-null  object 
 5   review_id          556871 non-null  object 
 6   review_date        556593 non-null  object 
 7   review_text        556871 non-null  object 
 8   review_language    556871 non-null  object 
 9   review_rating      278435 non-null  float64
 10  hotel_rating_mean  556382 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 51.0+ MB


In [4]:
vectorizer = TfidfVectorizer()

In [5]:
X = vectorizer.fit_transform(df['review_text'])

In [6]:
from scipy.sparse import csr_matrix

# Assuming your_sparse_matrix is the sparse matrix you have
# Splitting the matrix into two parts based on row indices
X_train = X[:278435]
X_test = X[278435:]

In [8]:
X_train = hstack([X_train, df_train.loc[:, 'hotel_rating_mean'].values.reshape(-1, 1)])
X_test = hstack([X_test, df_test.loc[:, 'hotel_rating_mean'].values.reshape(-1, 1)])

In [9]:
y_train = df_train['review_rating']

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.2, random_state=42)

In [11]:
# XGBoost regression model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

In [12]:
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [13]:
pred = xg_reg.predict(X_valid)

In [14]:
# Evaluating the model
rmse = mean_squared_error(y_valid, pred, squared=False)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1.2327800394713297


In [15]:
pred_test = xg_reg.predict(X_test)

In [16]:
submission = pd.DataFrame()
submission['review_id'] = df_test['review_id']
submission['review_rating'] = pred_test

submission.head()    

,review_id,review_rating
0,9p62eIN5NkEadRe7,7.817492
1,WfQozacJ1lzXMFQg,9.447377
2,oahzyhSR3ZNHJBuI,9.228675
3,lcO2pUMVQ09RmbHZ,6.634960
4,DmMu1z2SozPxykUS,8.797159


In [17]:
submission.to_csv(r'submission/submission_4.csv', index=False)